<a href="https://colab.research.google.com/github/Benendead/LSTMjazz/blob/master/Related_Works.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Source 1: [Text-based LSTM networks for Automatic Music Composition](https://arxiv.org/pdf/1604.05358.pdf)

## Introduction

This paper used text-based LSTMs for music composition. They found that word-RNNs could learn chord progressions and drum tracks, while char-RNNs could only succeed with chord progressions.

Their work attempted to generate jazz chord progressions and rock drum tracks. Their work is notable for two reasons:
1. Their LSTMs were designed to learn from text data as opposed to music symbols or numeric values.
2. They used a larger dataset than previous work, which allows learning of more complex chord progressions.

## Architecture

Char-RNNs are RNNs with character-based learning, as opposed to the conventional word-based learning. These return a vector corresponding to a character, while word-RNNs predict vectors corresponding to unique words. The merits of char-RNNs include:
1. Minimal assumptions are made - there are no constraints on the form of the text representation. Can RNNs learn musical information with such a weak assumption?
2. Fewer characters means fewer states, which reduces computational costs. Their chord vocabulary had 1,259 "words" yet this reduces to 39 characters.

It's worth noting that char-RNNs need to form longer-term dependencies, as the sequence becomes longer. LSTMs help with this, but the trade-off does have downsides we'll see later.

They used two LSTM layers, each with 512 hidden units. According to [reddit](https://www.reddit.com/r/MachineLearning/comments/87djn7/d_what_is_meant_by_number_of_hidden_units_in_an/), this 512 refers to the dimensionality of the hidden state ([Keras documentation](https://keras.io/layers/recurrent/) supports this answer). Dropout of 0.2 was added after each LSTM layer.

The model was put together using *Keras*. Loss function was categorical cross-entropy and optimizer was ADAM. This performed just as well as SGD with Nestrov momentum.

The prediction is stochastic in that each prediction for time $t$, the network outputs probabilities for every state. The system also had a diversity parameter $\alpha$ in the prediction stage, which suppresses $(\alpha < 1)$ or encourages $(\alpha < 1)$ the diversity of prediction by re-weighting the probabilities. This is done using the formula:  
$\hat{p}_i=e^{log(p_i)/\alpha}$, where $p_i$ is the probability for the $i$ states.  
Finally, one of the states is selected using the re-weighted probabilities.

The same size and number of layers was kept for both the char- and word-RNN experiments. The [code](https://github.com/keunwoochoi/lstm_real_book) is available online. I'll look through it once I'm done with the paper.

## Case Study: Chord Progressions

### Representation

They avoid vector representations of the chords and instead use only the text representations. They filled in each chord for all quarter notes during the chord and that's the dataset. Ex:  

F:9 F:9 F:9 F:9 D:min7
D:min7 G:9 G:9 C:maj C:maj
F:9 F:9 C:maj C:maj C:maj
C:maj

Their dataset included 2,486 scores from real and fake books. They transposed everything to C and filled in the duration of chords per quarter note. All song data was appended using _START_ and _END_ flags.

Even when transposed to C, only 867 scores end in C:maj, 489 in G:7, 186 C:maj6, 52 F:maj, and 1,252 in other chords. There were 1,259 unique chords in the training data, which gave the word-RNN a vocab size of 1,259. There were 39 distinct characters, 539,609 chords, and 3,531,261 characters total.

### Results

The system was set to output a chord progression for each diversity parameter after every iteration. Both the word- and char-RNNs showed well structured results. They learned local structures of chords and bars as well as the _START_ and _END_ tags.

After enough training, both results showed chord progressions from real jazz grammar. In  char-RNN, this included ii-V-I, passing chords, modal interchange chords, and substitutions (B:7 for F:7). For word-RNN, these included modal interchanges, circle of fifths (Eb:sus - Gb:maj6 - B:maj7), and descending bass.

The approaches differed slightly in output, as the word-RNN appeared to prefer more conventional progressions than in char-RNN. This may be caused by the different effective lengths of the approaches, as the char-RNN effective has a shorter memory span.

## Conclusion

They used text-based LSTMs to generate chord progressions based on jazz real books. Both word-RNNs and char-RNNs worked in this use case. Their usage of a diversity parameter gives composers a useful tool to dial in desired effects.

# Source 2: [Charlie Parker's Omnibook Data](https://members.loria.fr/KDeguernel/omnibook/)

Just a note that this is the dataset we used. Citation below:

Using Multidimensional Sequences For Improvisation In The OMax Paradigm
Ken Déguernel, Emmanuel Vincent, Gérard Assayag
13th Sound and Music Computing Conference, Aug 2016, Hamburg, Germany. 〈http://quintetnet.hfmt-hamburg.de/SMC2016/〉

# Source 3: [A First Look at Music Composition using LSTM Recurrent Neural Networks](http://people.idsia.ch/~juergen/blues/IDSIA-07-02.pdf)

**Abstract**  
Music composed by RNNs typically suffers from a lack of global structure. Note-by-note transitions or phrase reproduction might succeed, but overall musical form eludes RNNs' grasp. Luckily, LSTMs can overcome the long-term dependency issues and create music that's surprisingly pleasing in the blues style.

## Introduction

A most basic attempt a composition with RNNs might consider single-step predictions. This can then be seeded with just one example and attempt to compose from there. To state the obvious, feed-forward networks coud never perform such a task, as they can't store information about the past. RNNs overcome this basic limitation, at least.

RNNs still struggle with this task, unfortunately. Mozer, in 1994, wrote of his RNN's compositions that “While the
local contours made sense, the pieces were not musically coherent, lacking thematic structure and having minimal phrase structure and rhythmic organization." This problem likely links to vanishing gradients, as in both BPTT and RTRL error flow tends to explode or vanish.

Of course music requires this overall contour Mozer mentions. At the most basic form, early rock-and-roll might consist of four-bar phrases which easily become 32 events or more as the time step is defined as eighth notes. Before this paper, Mozer's single-note melodies were the most relevant work. Mozer had used RNNs with BPTT, probabilistic output values, and a psychologically realistic encoding that gave bias towards chromatically and harmonically related notes. He also encoded events in fewer time steps by treating all notes as one time step. Even still, the architecture failed to capture global musical structure.

Mozer suggested that for a note-by-note method to work, it requires a network which can induce structure at multiple levels. This paper offers that network.



## An LSTM Music Composer

### LSTM Overview

To summarize LSTMs, they're designed to obtain constant error flow through time so that nothing explodes or vanishes. LSTMs use linear units called Constant Error Carousels (CECs), apparently. Really the CEC is just another way to represent the continuing cells we otherwise get when we consider time steps.

The gates which control information into the CEC include:
1. Multiplicative **Input Gate** - Learns to protect the information passed into the CEC by rejecting irrelevant inputs.
2. Multiplicative **Output Gate** - Learns to protect other areas from currently irrelevant memory contents of the CEC.
3. **Forget Gate** - Learns to reset memory cells when their content is obsolete.

Learning is then done using a modified BPTT and a customized version of RTRL. That is, output units use BPTT, output gates use truncated BPTT, and the input and forget gates use truncated RTRL.

### Data Representation

This paper represented the data in simple local form, using one input per note, with 1 as on and 0 as off. In later experiments, they adjusted input units to have a mean of 0 and a standard deviation of 1 (???). This left it to the network to develop a bias towards chromatic or harmonic notes. Their reasons follow:
1. Implicitly multivoice and makes no distinction between chords and melodies. (They implemented chords simply by including them in the single input vector)
2. It's easy to generate probability distributions over the set of possible notes, as they can treat single notes as independent or dependent.

Time was represented as a single input vector representing one slice of real time. The stepsize of each time step of course can vary; they chose an eighth note as theirs. This is preferable for LSTMs, as the network needs to learn relative durations of notes as to create rhythm and coutning.

This representation does ignore two issues:
1. There's no indication where a note ends. Thus eight eighth notes in a row of the same notes are equivalent to four quarters or a whole note. One way to overcome this could be to decrease the stepsize of quantization and mark note endings with a zero (???).
2. A second method was suggested by [Todd in 1989](https://pdfs.semanticscholar.org/81e0/a57abde1bf2cc4b7cd772e0573e92069e8ef.pdf). This used special units in the network to indicate the beginning of notes. These authors were unsure how that might scale to multi-voice melodies.

The LSTM was given a range of 12 notes for the chords and 13 notes for melodies. These were simply two sections of the vector but otherwise were represented no differently.

### Training Data

This experiment used a form of 12-bar blues with a quantization step of 8 notes per bar. Each song was thus 96 time steps long. The same chords were used in every song, and they were inverted as to fit into the 12 note options for the chords. Experiment 1 used only the chords whereas Experiment 2 also included a single melody line based on the pentatonic scale.

Training melodies were constructed by concatenating bar-long segments of music by the first author to fit each chord. The datasets were then constructed by concatenating random samples from the $(n=2^{12}=4096)$ possible songs. The melodies included no rests and were only quarter notes.

## Experiment 1 - Learning Chords

Can an LSTM reproduce a musical chord structure? The motivation here is to ensure that the LSTM doesn't require melody.

### Network Topology and Hyperparameters

The chords used were the same 12-bar blues. The network had four cell blocks containing 2 cells each, fully connected to each other and to the input layer. The output layer was fully connected to all cells and the input layer. Forget, input, and output gate biases for the four blocks were set to -0.5, -1.0, -1.5, and -2.0. This causes the blocks to come online one by one. Output biases were set to 0.5, the learning rate 0.00001, and momentum rate 0.9.

Weights were burned after every timestep, as experiments showed that learning was faster if the network was reset after making one or a few gross errors. Resetting went as follows:  
On error, burn existing weights, reset input pattern and clear partial derivatives, activations, and cell states. This was similar to Gers and Schmidhuber's (2000) approach. The output function was the logistic sigmoid with range [0,1].

### Training/Testing

The goal was to predict the probability for a given note to be on or off. For predicting probabilities, RMSE is not appropriate and thus the network was trained using cross-entropy as the objective function. The error function $E_i$ for output activation $y_i$ and target $t_i$ was:  
$E_i=-t_i*ln(y_i)-(1-t_i)*ln(1-y_i).$

This gives a $\delta$ term at the output layer of $(t_i-y_i)$. See [Joost and Schiffmann](https://www.worldscientific.com/doi/abs/10.1142/S0218488598000100) (1998) for what exactly this means. Note that they treat the outputs as statistically inpedendent of one another. Even if this assumption is untrue, it allows the network to predict chords and melodies in parallel (as well as multi-voice melodies).

The testing started the network with the inputs from the first timestep and used network predictions for ensuing time steps. The decision threshold for chord notes was 0.5. Training ended once the network spit back the correct chord sequence in entirety.

### Results

In the end, the network could accomplish this task with a variety of learning rates and momentum rates. The network was able to generate continuing cycles of the progression as well. This result is not surprising, honestly. The learning time took anywhere from 15 to 45 minutes on a 1Ghz Pentium.